In [ ]:
import pandas as pd

# Paths to the input CSV files
athletes_file = "olympic_athletes.csv"
medals_file = "olympic_medals.csv"
results_file = "olympic_results.csv"

# Output files
nodes_output_file = "medal_tally_nodes_with_medals.csv"
edges_output_file = "medal_tally_edges.csv"

# Step 1: Create medal_tally_nodes_with_medals.csv
def create_medal_tally_nodes(medals_file, output_file):
    # Load the medals data
    medals_df = pd.read_csv(medals_file)

    # Aggregate medal counts by country
    aggregated_medals = medals_df.groupby('country_name').agg(
        total_medals=('medal_type', 'count'),
        gold=('medal_type', lambda x: (x == 'GOLD').sum()),
        silver=('medal_type', lambda x: (x == 'SILVER').sum()),
        bronze=('medal_type', lambda x: (x == 'BRONZE').sum())
    ).reset_index()

    # Save the results to the output file
    aggregated_medals.to_csv(output_file, index=False)
    print(f"Created {output_file}")

# Step 2: Create medal_tally_edges.csv
def create_medal_tally_edges(results_file, medals_file, output_file):
    # Load results and medals data
    results_df = pd.read_csv(results_file)

    # Aggregate competition participation by country and event
    event_participation = results_df.groupby(['event_title', 'country_name']).size().reset_index(name='participation_count')

    # Create edges between countries based on shared events
    edges = event_participation.merge(event_participation, on='event_title')
    edges = edges[edges['country_name_x'] != edges['country_name_y']]  # Exclude self-loops

    # Aggregate rivalry weights based on shared event participation
    edges = edges.groupby(['country_name_x', 'country_name_y']).agg(
        weight=('participation_count_x', 'sum')
    ).reset_index()

    # Rename columns for clarity
    edges = edges.rename(columns={'country_name_x': 'source', 'country_name_y': 'target'})

    # Save the results to the output file
    edges.to_csv(output_file, index=False)
    print(f"Created {output_file}")

# Run the scripts
create_medal_tally_nodes(medals_file, nodes_output_file)
create_medal_tally_edges(results_file, medals_file, edges_output_file)


Created medal_tally_nodes_with_medals2.csv
Created medal_tally_edges2.csv


In [ ]:
import pandas as pd

def generate_cumulative_medals(results_file, medals_file, athletes_file, output_file):
    # Load the datasets
    results_df = pd.read_csv(results_file)
    medals_df = pd.read_csv(medals_file)
    athletes_df = pd.read_csv(athletes_file)

    # Merge results with medals to get country information
    merged_df = results_df.merge(medals_df, on="event_id", how="inner")
    merged_df = merged_df.merge(athletes_df, on="athlete_id", how="inner")

    # Ensure the year column exists
    if "year" not in merged_df.columns:
        merged_df["year"] = merged_df["date"].str[:4].astype(int)  # Extract year from date

    # Aggregate medal counts by year and country
    medals_by_year = (
        merged_df.groupby(["year", "country_name"])
        .size()
        .reset_index(name="medal_count")
    )

    # Sort by year and country
    medals_by_year = medals_by_year.sort_values(by=["year", "country_name"])

    # Compute cumulative medal counts
    medals_by_year["cumulative_total_medals"] = (
        medals_by_year.groupby("country_name")["medal_count"].cumsum()
    )

    # Save to CSV
    medals_by_year.rename(columns={"country_name": "country"}, inplace=True)
    medals_by_year.to_csv(output_file, index=False)
    print(f"Generated cumulative medals data saved to {output_file}")

# File paths (update these paths as necessary)
results_file = "olympic_results.csv"
medals_file = "olympic_medals.csv"
athletes_file = "olympic_athletes.csv"
output_file = "cumulative_olympic_medals.csv"

# Generate the cumulative medals file
generate_cumulative_medals(results_file, medals_file, athletes_file, output_file)
